In [54]:
import pandas as pd
import requests
import csv
import os
import zipfile
import io
import glob

## Fyers.in

In [7]:
fyers_01 = pd.read_csv("https://public.fyers.in/sym_details/NSE_CD.csv")
fyers_02 = pd.read_csv("https://public.fyers.in/sym_details/NSE_FO.csv")
fyers_03 = pd.read_csv("https://public.fyers.in/sym_details/NSE_CM.csv")
fyers_04 = pd.read_csv("https://public.fyers.in/sym_details/BSE_CM.csv")
fyers_05 = pd.read_csv("https://public.fyers.in/sym_details/BSE_FO.csv")
fyers_06 = pd.read_csv("https://public.fyers.in/sym_details/MCX_COM.csv")

In [12]:
fyers_01.columns , fyers_02.columns , fyers_03.columns, fyers_04.columns , fyers_05.columns , fyers_06.columns

(Index(['101224102910107', 'JPYINR 24 Oct 29 65.5 CE', '19', '1', '0.0025',
        'Unnamed: 5', '0900-1700|1815-1915:', '2024-10-28', '1730185200',
        'NSE:JPYINR24OCT65.5CE', '10', '12', '10107', 'JPYINR', '27', '65.5',
        'CE', '101200000027', 'None', '0', '0.0'],
       dtype='object'),
 Index(['101124102935012', 'FINNIFTY 24 Oct 29 FUT', '11', '25', '0.05',
        'Unnamed: 5', '0915-1530|1815-1915:', '2024-10-28', '1730196000',
        'NSE:FINNIFTY24OCTFUT', '10', '11.1', '35012', 'FINNIFTY', '26037',
        '-1.0', 'XX', '101000000026037', 'None', '0', '0.0'],
       dtype='object'),
 Index(['10100000001', 'GOLDSTAR POWER LIMITED', '0', '11250', '0.05',
        'INE405Y01021', '0915-1530|1815-1915:', '2024-10-28', 'Unnamed: 8',
        'NSE:GOLDSTAR-SM', '10', '10.1', '1', 'GOLDSTAR', '1.1', '-1.0', 'XX',
        '10100000001.1', 'None', '0.1', '0.0'],
       dtype='object'),
 Index(['12100000001', 'SENSEX-INDEX', '10', '0', '0.01', 'Unnamed: 5',
        '0915-1530

In [16]:
data = pd.concat([fyers_01, fyers_02, fyers_03, fyers_04, fyers_05, fyers_06], ignore_index=True)
data.columns = data.columns.str.strip()

In [17]:
def classify_instrument(name):
    if isinstance(name, str):  
        if 'FUT' in name:
            return 'Futures'
        elif 'CE' in name or 'PE' in name:
            return 'Options'
        else:
            return 'Cash'
    else:
        return 'Unknown'  


In [19]:
data['Type'] = data.iloc[:, 1].apply(classify_instrument)

In [20]:
cash_data = data[data['Type'] == 'Cash']
futures_data = data[data['Type'] == 'Futures']
options_data = data[data['Type'] == 'Options']

In [21]:
os.makedirs('fyers', exist_ok=True)

In [22]:
cash_data.to_csv('fyers/cash_data.csv', index=False)
futures_data.to_csv('fyers/futures_data.csv', index=False)
options_data.to_csv('fyers/options_data.csv', index=False)

### Angel One

In [23]:
url_angel_one = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = requests.get(url_angel_one)
response.raise_for_status()  

In [26]:
data_angel_one  = response.json()
df_angel_one  = pd.DataFrame(data_angel_one )
df_angel_one.columns

Index(['token', 'symbol', 'name', 'expiry', 'strike', 'lotsize',
       'instrumenttype', 'exch_seg', 'tick_size'],
      dtype='object')

In [34]:
def classify_instrument_angel(row):
    """Classify instruments into Cash, Futures, or Options."""
    instrument_type = row['instrumenttype']
    symbol = row['symbol']

    if instrument_type in ['OPTSTK', 'OPTIDX']:
        return 'Options'
    elif instrument_type in ['FUTSTK', 'FUTIDX']:
        return 'Futures'
    else:
        return 'Cash'

In [35]:
df_angel_one['Type'] = df_angel_one.apply(classify_instrument_angel, axis=1)

In [36]:
cash_data = df_angel_one[df_angel_one['Type'] == 'Cash']
futures_data = df_angel_one[df_angel_one['Type'] == 'Futures']
options_data = df_angel_one[df_angel_one['Type'] == 'Options']

In [37]:
os.makedirs('angel_one', exist_ok=True)

cash_data.to_csv('angel_one/cash_data.csv', index=False)
futures_data.to_csv('angel_one/futures_data.csv', index=False)
options_data.to_csv('angel_one/options_data.csv', index=False)


### icici 

In [8]:

icici_url = "https://directlink.icicidirect.com/NewSecurityMaster/SecurityMaster.zip"
os.makedirs('raw_icici', exist_ok=True)


In [9]:

def extract_zip_and_convert(url, extract_to='raw_icici'):
    response = requests.get(url)
    response.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(path=extract_to)
        for filename in z.namelist():
            print(filename)
    for filename in os.listdir(extract_to):
        if filename.endswith('.txt'):
            txt_path = os.path.join(extract_to, filename)
            with open(txt_path, 'r') as file:
                csv_content = file.read().replace('|', ',')

            new_filename = os.path.splitext(filename)[0] + '.csv'
            csv_path = os.path.join(extract_to, new_filename)
            with open(csv_path, 'w') as csv_file:
                csv_file.write(csv_content)

In [10]:
extract_zip_and_convert(icici_url)

Extracted files:
BSEScripMaster.txt
CDNSEScripMaster.txt
FOBSEScripMaster.txt
FONSEScripMaster.txt
NSEScripMaster.txt
Converted BSEScripMaster.txt to BSEScripMaster.csv
Converted CDNSEScripMaster.txt to CDNSEScripMaster.csv
Converted FOBSEScripMaster.txt to FOBSEScripMaster.csv
Converted FONSEScripMaster.txt to FONSEScripMaster.csv
Converted NSEScripMaster.txt to NSEScripMaster.csv


In [13]:
for file in os.listdir('raw_icici'):
    print(file)
csv_files = [f for f in os.listdir('raw_icici') if f.endswith('.csv')]
if csv_files:
    dfs = [pd.read_csv(os.path.join('raw_icici', f), on_bad_lines='skip', low_memory=False) for f in csv_files]
    df_icici = pd.concat(dfs, ignore_index=True)
    df_icici.to_csv('raw_icici/icici_master_scrip.csv', index=False)
else:
    print("No CSV files found in the 'icici' directory.")

BSEScripMaster.csv
BSEScripMaster.txt
CDNSEScripMaster.csv
CDNSEScripMaster.txt
FOBSEScripMaster.csv
FOBSEScripMaster.txt
FONSEScripMaster.csv
FONSEScripMaster.txt
icici_master_scrip.csv
NSEScripMaster.csv
NSEScripMaster.txt


In [19]:
df_icici = pd.read_csv("raw_icici\icici_master_scrip.csv",low_memory=False)

In [20]:
print(f"Columns: {df_icici.columns}")

Columns: Index(['Token', 'ShortName', 'Series', 'CompanyName', 'TickSize', 'LotSize',
       'ScripCode', 'MarketLot', 'BCastFlag', 'AVMBuyMargin',
       ...
       'NormalMarketStatus.1', 'OddLotMarketStatus.1', 'SpotMarketStatus.1',
       'AuctionMarketStatus.1', 'NormalMarketEligibility.1',
       'OddLotlMarketEligibility', 'SpotMarketEligibility.1',
       'AuctionlMarketEligibility', 'MarginPercentage.1', 'ExchangeCode.1'],
      dtype='object', length=155)


In [25]:

def classify_instrument_icici(row):
    """Classify instruments into Cash, Futures, or Options."""
    instrument_name = row['InstrumentName']
    symbol = row['Symbol'] if 'Symbol' in row else row['ScripName']  
    if instrument_name in ['OPTSTK', 'OPTIDX']:
        return 'Options'
    elif instrument_name in ['FUTSTK', 'FUTIDX']:
        return 'Futures'
    else:
        return 'Cash'

In [26]:
df_icici['Type'] = df_icici.apply(classify_instrument_icici, axis=1)
cash_data = df_icici[df_icici['Type'] == 'Cash']
futures_data = df_icici[df_icici['Type'] == 'Futures']
options_data = df_icici[df_icici['Type'] == 'Options']

In [27]:
os.makedirs('icici', exist_ok=True)
cash_data.to_csv('icici/cash_data.csv', index=False)
futures_data.to_csv('icici/futures_data.csv', index=False)
options_data.to_csv('icici/options_data.csv', index=False)

### Kotak

In [ ]:
kotak_urls = {
    "cash": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_Cash_01_04_2022.txt",
    "futures": "https://preferred.kotaksecurities.com/security/production/TradeApiInstruments_FNO_01_04_2022.txt"
}

In [48]:
df_cash = pd.read_csv(kotak_urls['cash'], delimiter='|')
df_futures = pd.read_csv(kotak_urls['futures'], delimiter='|')

df_kotak = pd.concat([df_cash, df_futures], ignore_index=True)

In [49]:
def classify_instrument(row):
    instrument_type = row['instrumentType'].strip() if isinstance(row['instrumentType'], str) else ''
    option_type = row['optionType'].strip() if isinstance(row['optionType'], str) else ''
    if option_type:
        return 'Options'
    elif instrument_type in ['FUTSTK', 'FUTIDX']:
        return 'Futures'
    else:
        return 'Cash'

In [50]:
df_kotak['Type'] = df_kotak.apply(classify_instrument, axis=1)

In [51]:
cash_data = df_kotak[df_kotak['Type'] == 'Cash']
futures_data = df_kotak[df_kotak['Type'] == 'Futures']
options_data = df_kotak[df_kotak['Type'] == 'Options']

In [52]:
os.makedirs('kotak', exist_ok=True)
cash_data.to_csv('kotak/cash_data.csv', index=False)
futures_data.to_csv('kotak/futures_data.csv', index=False)
options_data.to_csv('kotak/options_data.csv', index=False)

### Zerodha

In [60]:

def fetch_zerodha_data():
    url_zerodha = "https://api.kite.trade/instruments"
    filename = "zerodha.csv"
    path = f"raw_zerodha/{filename}"

    print(f"Fetching data from {url_zerodha}...")

    response = requests.get(url_zerodha)
    response.raise_for_status()

    os.makedirs('raw_zerodha', exist_ok=True)
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file)
        for line in response.text.splitlines():
            writer.writerow(line.split(','))
    print(f"done!")
fetch_zerodha_data()


Fetching data from https://api.kite.trade/instruments...
done!


In [61]:
zerodha_data = pd.read_csv('raw_zerodha/zerodha.csv')
zerodha_data.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,265969926,1038945,EURINR24NOVFUT,"""EURINR""",0,2024-11-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
1,268449030,1048629,EURINR24DECFUT,"""EURINR""",0,2024-12-27,0.0,0.0025,1,FUT,BCD-FUT,BCD
2,269248262,1051751,EURINR25JANFUT,"""EURINR""",0,2025-01-29,0.0,0.0025,1,FUT,BCD-FUT,BCD
3,270018310,1054759,EURINR25FEBFUT,"""EURINR""",0,2025-02-26,0.0,0.0025,1,FUT,BCD-FUT,BCD
4,270682630,1057354,EURINR25MARFUT,"""EURINR""",0,2025-03-27,0.0,0.0025,1,FUT,BCD-FUT,BCD


In [63]:
zerodha_data.columns

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')

In [66]:
os.makedirs('zerodha', exist_ok=True)

df_cash = zerodha_data[zerodha_data['instrument_type'] == 'EQ'] 
df_futures = zerodha_data[zerodha_data['instrument_type'] == 'FUT'] 
df_options = zerodha_data[zerodha_data['instrument_type'] == 'OPT'] 



In [67]:
df_cash.to_csv('zerodha/cash_data.csv', index=False)
df_futures.to_csv('zerodha/futures_data.csv', index=False)
df_options.to_csv('zerodha/options_data.csv', index=False)

### Final Merging analysis

In [10]:
import pandas as pd
import os

# Define brokers and data types
brokers = ['kotak', 'angel_one', 'icici', 'fyers', 'zerodha']
data_types = ['cash', 'futures', 'options']

# Initialize a nested dictionary to store the mapped data
data_dict = {broker: {} for broker in brokers}

# Load all CSV files into the data_dict
for broker in brokers:
    for data_type in data_types:
        file_path = f'./{broker}/{data_type}_data.csv'
        if os.path.exists(file_path):
            try:
                data_dict[broker][data_type] = pd.read_csv(file_path)
                print(f"Loaded: {file_path}")
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
        else:
            print(f"File {file_path} not found.")

# Example of how to access the mapped data:
print("\n--- Example Data Access ---")
for broker, data in data_dict.items():
    for data_type, df in data.items():
        print(f"\nBroker: {broker}, Data Type: {data_type}")
        print(df.head())  # Display the first few rows of the dataframe

# Optional: Save the individual broker data into separate subfolders (optional)
def save_individual_csv_files(data_dict):
    for broker, data in data_dict.items():
        for data_type, df in data.items():
            output_path = f'./mapped_output/{broker}_{data_type}_data.csv'
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"Saved {output_path}")

# Call the function to save (if needed)
# save_individual_csv_files(data_dict)

print("\nData mapping completed successfully!")


Loaded: ./kotak/cash_data.csv
Loaded: ./kotak/futures_data.csv
Loaded: ./kotak/options_data.csv
Loaded: ./angel_one/cash_data.csv
Loaded: ./angel_one/futures_data.csv
Loaded: ./angel_one/options_data.csv


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2992\3597312919.py:17: DtypeWarning: Columns (0,1,2,3,11,12,25,33,34,35,37,66,86,95,96,97,104,154) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict[broker][data_type] = pd.read_csv(file_path)


Loaded: ./icici/cash_data.csv
Loaded: ./icici/futures_data.csv
Loaded: ./icici/options_data.csv
Loaded: ./fyers/cash_data.csv
Loaded: ./fyers/futures_data.csv
Loaded: ./fyers/options_data.csv
Loaded: ./zerodha/cash_data.csv
Loaded: ./zerodha/futures_data.csv
Loaded: ./zerodha/options_data.csv

--- Example Data Access ---

Broker: kotak, Data Type: cash
   instrumentToken instrumentName                  name  lastPrice  expiry  \
0             9516      20MICRONS        20 Microns Ltd      73.85       0   
1            43865      3IINFOLTD       3i Infotech Ltd      51.25       0   
2              891        3MINDIA          3M India Ltd   19697.30       0   
3              607         3PLAND  3P Land Holdings Ltd      14.50       0   
4            13140         5PAISA    5Paisa Capital Ltd     344.70       0   

   strike  tickSize  lotSize instrumentType segment exchange          isin  \
0     0.0      0.05        1             EQ    CASH      NSE  INE144J01027   
1     0.0      0.05 

In [14]:
import pandas as pd
import os

# Define brokers and data types
brokers = ['kotak', 'angel_one', 'icici', 'fyers', 'zerodha']
data_types = ['cash', 'futures', 'options']

# Initialize an empty list to collect all data for the master CSV
master_data = []

# Load all CSV files into the master_data list
for broker in brokers:
    for data_type in data_types:
        file_path = f'./{broker}/{data_type}_data.csv'
        
        if os.path.exists(file_path):
            try:
                # Load the CSV file
                df = pd.read_csv(file_path)
                print(f"Loaded: {file_path}")

                # Add broker and data type as new columns
                df['Broker'] = broker
                df['Data_Type'] = data_type

                # Append the data to the master list
                master_data.append(df)

            except Exception as e:
                print(f"Error loading {file_path}: {e}")
        else:
            print(f"File {file_path} not found.")

# Combine all individual DataFrames into a single DataFrame
if master_data:
    master_df = pd.concat(master_data, ignore_index=True)
    
    # Save the master DataFrame as a CSV
    master_csv_path = 'master_data.csv'
    master_df.to_csv(master_csv_path, index=False)
    print(f"\nMaster CSV saved at: {master_csv_path}")
else:
    print("No data to save to the master CSV.")

# Optional: Save individual broker files into mapped_output folder
def save_individual_csv_files(data_dict):
    for broker, data in data_dict.items():
        for data_type, df in data.items():
            output_path = f'./mapped_output/{broker}_{data_type}_data.csv'
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"Saved {output_path}")

print("\nData mapping and aggregation completed successfully!")


Loaded: ./kotak/cash_data.csv
Loaded: ./kotak/futures_data.csv
Loaded: ./kotak/options_data.csv
Loaded: ./angel_one/cash_data.csv
Loaded: ./angel_one/futures_data.csv
Loaded: ./angel_one/options_data.csv


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2992\1257365914.py:19: DtypeWarning: Columns (0,1,2,3,11,12,25,33,34,35,37,66,86,95,96,97,104,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Loaded: ./icici/cash_data.csv
Loaded: ./icici/futures_data.csv
Loaded: ./icici/options_data.csv
Loaded: ./fyers/cash_data.csv
Loaded: ./fyers/futures_data.csv
Loaded: ./fyers/options_data.csv
Loaded: ./zerodha/cash_data.csv
Loaded: ./zerodha/futures_data.csv
Loaded: ./zerodha/options_data.csv

Master CSV saved at: master_data.csv

Data mapping and aggregation completed successfully!
